# モデル定義用 Notebook

## ファイルのダウンロード

In [ ]:
# COSのcredentaial情報
# 各自の設定に差し替えて下さい
credentials = {
    'IBM_API_KEY_ID': 'xxxx',
    'IAM_SERVICE_ID': 'xxxx',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'xxxx',
    'FILE': 'xxxx.xxx'
}

In [2]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [3]:
# download zip file from COS
zipfile = 'model-kit-cnn2.zip'
cos.download_file(Bucket=credentials['BUCKET'],Key=zipfile,Filename=zipfile)

In [4]:
!ls -l

total 16
-rw-r----- 1 dsxuser dsxuser 10700 May 22 07:11 gosales-101-500.csv
-rw-r----- 1 dsxuser dsxuser  2342 May 22 09:54 model-kit-cnn2.zip


## Watson ML 管理クライアント

In [6]:
import urllib3, requests, json, base64, time, os, warnings
warnings.filterwarnings('ignore')

In [7]:
# Watson Machine LearningのCredntial情報
# 各自の設定に差し替えて下さい。
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "xxxxx",
  "password": "xxxxx",
  "instance_id": "xxxxx"
}

In [9]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

1.0.135


## Training定義

In [15]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "CNN CIFAR-10 KIT v3",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: 'python3 dlaas_train.py train_model_kit2.py CNN-CIFAR10-HPO.h5'   
            }

In [16]:
# 定義ファイルの登録
definition_details = client.repository.store_definition(zipfile, model_definition_metadata)

In [17]:
import json
print(json.dumps(definition_details, indent=2))

{
  "entity": {
    "name": "CNN CIFAR-10 KIT v3",
    "training_definition_version": {
      "content_url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/training_definitions/255e537c-3c0f-4766-95b4-a0f3f0569c13/versions/8233159a-015e-4acd-8b03-d027b1ffa83c/content",
      "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/training_definitions/255e537c-3c0f-4766-95b4-a0f3f0569c13/versions/8233159a-015e-4acd-8b03-d027b1ffa83c",
      "guid": "8233159a-015e-4acd-8b03-d027b1ffa83c"
    },
    "command": "python3 dlaas_train.py train_model_kit2.py CNN-CIFAR10-HPO.h5",
    "framework": {
      "name": "tensorflow",
      "version": "1.5",
      "runtimes": [
        {
          "name": "python",
          "version": "3.5"
        }
      ]
    }
  },
  "metadata": {
    "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/training_definitions/255e537c-3c0f-4766-95b4-a0f3f0569c13",
    "guid": "255e537c-3c0f-4766-95b4-a0f3f0569c13",
    "created_at": "2018-05-22T10:03:26.020Z"

In [26]:
# 定義ファイルの一覧
client.repository.list_definitions()

------------------------------------  -------------------  ------------------------  ----------
GUID                                  NAME                 CREATED                   FRAMEWORK
5ea80ee6-272b-444d-a3fe-8ca12aab3d87  TENT multiclass      2018-04-30T01:32:32.595Z  wml
54441537-0320-4f2a-9df8-1222964b5cd8  Iris                 2018-04-30T02:24:36.416Z  wml
441bd817-e614-4b73-9382-3351870ac14f  Regression-Sample    2018-05-03T16:03:59.937Z  wml
c635c7a5-0d28-453d-a29c-7a19d2eef275  CNN CIFAR10 aka v5   2018-05-11T01:04:12.188Z  tensorflow
7a2a1762-5938-4f82-b9a1-1c2ebf0cc052  CNN CIFAR10 HPO      2018-05-11T06:43:15.936Z  tensorflow
088683ac-dc29-42f6-a140-d23f301583c2  CNN-CIFAR-10-KIT-2   2018-05-22T05:15:04.347Z  tensorflow
255e537c-3c0f-4766-95b4-a0f3f0569c13  CNN CIFAR-10 KIT v3  2018-05-22T10:03:26.020Z  tensorflow
------------------------------------  -------------------  ------------------------  ----------


In [25]:
client.repository.delete('0b120a1b-b3c2-4aa3-b6c6-1985f96a2e5f')